# Kubernetes

# Kubernetes Version (Current)

In [6]:
#!pip install ipynb
from ipynb.fs.full.facturxWidget import *

from ipynb.fs.full.kubernetesTool import *

## addPeer

In [238]:
import kubernetesTool
import subprocess

class KubernetesPeer: 
    def __init__(self, name='',prefix='peer', hostname='kubernetes.research.dev.seeburger.de',ca_country='DE',ca_province='Baden',ca_locality='Bretten'):
        self.name = name

        self.addPeer(hostname,ca_country,ca_province,ca_locality)
        
        self.pod_name = kubernetesTool.getPod(name)
        self.listen_port = kubernetesTool.getPort(peername=name, name = prefix + '-listen')
        self.gossip_port = kubernetesTool.getPort(peername=name, name = prefix + '-gossip')
                
    def addPeer(self, hostname='kubernetes.research.dev.seeburger.de',ca_country='DE',ca_province='Baden',ca_locality='Bretten', prefix='peer'):  
        peername = self.name
        #if peername in peers.keys():
        #    return peers[peername]
        # Create configuration for new peer
        executeLocalCmd(['/bin/bash','-c','cd /home/jovyan/scray-ledger/containers/ && /home/jovyan/scray-ledger/containers/configure-deployment.sh -n ' + peername])
        # Start service
        executeKubectlCmd(['apply', '-f', '/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer-service.yaml'])
        self.listen_port = kubernetesTool.getPort(peername=peername, name = prefix + '-listen')
        self.gossip_port = kubernetesTool.getPort(peername=peername, name = prefix + '-gossip')
        # Create peer configuration
        res = self.createConfig(hostname=hostname,
                                ca_country=ca_country,ca_province=ca_province,ca_locality=ca_locality)
        # Start new peer
        executeKubectlCmd(['apply', '-f','/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer.yaml'])
        #peer = kubernetespeer(name=peername)
        #peers[peername] = self
        #return peer        
        
    def addPeerFromKubernetes():
        peer= kubernetespeer(self.name)
        peers[self.name] = peer
        
    def getCertificate(self):
        try:
            return str(subprocess.check_output(['cat', self.crt]))[2:-3]   
        except Exception as e:
            return str(e)    
        
    def createConfig(self,hostname='kubernetes.research.dev.seeburger.de' ,ca_country='DE',ca_province='Baden',ca_locality='Bretten'):
        try:  
            peername=self.name
            peer_listen_port=self.listen_port
            #peer_gossip_port=self.gossip_port
            peer_gossip_port=self.listen_port
            address = 'peer0.' + peername  + '.' +  hostname + ':'
            output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'create','configmap','hl-fabric-peer-' + peername,
                                              '--from-literal=hostname=' + peername  + '.' + hostname,
                                              '--from-literal=org_name=' + peername,
                                              '--from-literal=data_share=hl-fabric-data-share-service:80',
                                              '--from-literal=ca_country='  + ca_country,
                                              '--from-literal=ca_province=' + ca_province,
                                              '--from-literal=ca_locality=' + ca_locality,
                                              '--from-literal=CORE_PEER_ADDRESS=' + address + peer_listen_port,
                                              '--from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=' + address + peer_gossip_port,
                                              '--from-literal=CORE_PEER_LOCALMSPID=' + peername + 'MSP'])
            return str(output)
        except Exception as e:
            return str(e)  

    
    def deleteConfig(self):
        try:  
            output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','configmap','hl-fabric-peer-' + self.name])
            return str(output)
        except Exception as e:
            return str(e)     
        
    def deleteDeployment(self):
        try:  
            output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','deployment', self.name])
            return str(output)
        except Exception as e:
            return str(e)           

    def deletePeer(self):
        self.deleteDeployment()
        self.deleteConfig()        
        
    def joinHyperledger(self,orderer_ip=None,orderer_hostname=None,orderer_port=None,channel_name=None,shared_fs_host=None,ext_peer_ip='10.15.136.41'):          
        cmd = toCmd(['/mnt/conf/peer_join.sh', orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip])
        print(cmd)
        return executePeerCmd(self.pod_name, cmd)

    def endorse_peer(self,orderer_ip='',channel_name='',shared_fs_host='',ext_peer_ip='10.15.136.41'):
        peername = self.name
        peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
        cmd = toCmd(['/mnt/conf/orderer/scripts/inform_existing_nodes.sh',orderer_ip,channel_name,self.name,shared_fs_host,ext_peer_ip,peer_host_name]) 
        #print(cmd)
        return executePeerCmd(orderer.pod_name,cmd,cli='scray-orderer-cli')

    def endorse_and_join(self):    
        res1 = self.endorse_peer(orderer_ip=orderer_ip,channel_name=channel_name,shared_fs_host='10.14.128.38:30080')   
        res2  = self.joinHyperledger(orderer_ip=orderer_ip,orderer_hostname=orderer_hostname,orderer_port=orderer_port,channel_name=channel_name,shared_fs_host=shared_fs_host,ext_peer_ip='10.15.136.41')
        return res1,res2        
        
    def installAndAproveChaincode(self,orderer,channel_name,cc_host='10.14.128.38',pkgid='',cc_hostname='asset-transfer-basic.org1.example.com',cc_label='basic_1.0',shared_fs='kubernetes.research.dev.seeburger.de:30080', sequence=1):
        #!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK
        #strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.orderer_intern), channel_name ]
        strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.listen_port), channel_name,pkgid,cc_hostname, cc_label,shared_fs, str(sequence) ]
        print(toCmd(strlist))
        return executePeerCmd(self.pod_name,toCmd(strlist)) 

    def commitChaincode(self,channel_name,sequence=1,pkgid=''):
        strlist = ['/mnt/conf/peer/cc_commit.sh', channel_name, str(sequence),pkgid ]
        print(toCmd(strlist))
        return executePeerCmd(self.pod_name,toCmd(strlist))                 
        
        
#class ordererpeer(kubernetes_peer):
class ordererpeer(object):
    def __init__(self, name='',prefix='orderer'):
        #super().__init__(name,prefix)
        self.name = ''
        self.pod_name = kubernetesTool.getPod(name)
        self.host = 'orderer.example.com'
        self.orderer_intern = 7050
        self.listen_port = kubernetesTool.getPort(peername=name, name = prefix + '-listen')
        
    # can change
    def get_ip(self, app='orderer-org1-scray-org'):
        return executeKubectlCmd(['get', 'pods','-l', 'app=' + app,'-o','jsonpath=\'{.items[*].status.podIP}\''])   
    

### Blockchain

In [240]:
class Blockchain(object):
    def __init__(self, name='orderer-org1-scray-org',prefix='orderer', channel_name='invoicing17'):
        self.peers = {}
        
        self.orderer = ordererpeer(name='orderer-org1-scray-org',prefix='orderer')
        self.orderer_ip=self.orderer.get_ip()
        self.orderer_hostname=self.orderer.host
        self.orderer_port= self.orderer.listen_port

        self.channel_name=channel_name
        self.shared_fs_host='10.14.128.38:30080'
 
    def addPeer(self,peername,hostname='kubernetes.research.dev.seeburger.de',ca_country='DE',ca_province='Baden',ca_locality='Bretten'):  
        if peername in self.peers.keys():
            return self.peers[peername]

        peer = KubernetesPeer(name=peername,hostname=hostname,ca_country=ca_country,ca_province=ca_province,ca_locality=ca_locality)
        self.peers[peername] = peer   

    def add_channel(self,name):
        self.channel_name=name
        cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', self.orderer.pod_name, '-c', 'scray-orderer-cli','--', '/bin/sh']
        cmd2 = ['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '30081']
        #cmd = toCmd(cmd1 + cmd2)
        ret = subprocess.run(cmd1 + cmd2, stdout = subprocess.PIPE, stderr=subprocess.PIPE)
        #return executePeerCmd(orderer.pod_name, cmd, cli='scray-orderer-cli')
        return ret.stdout.decode('ascii'), ret.stderr.decode('ascii')    
    
    def endorse_and_join_all_peers(self): 
        out2 = widgets.Output()
        with out2:
            for key, peer in self.peers.items():
                peer.endorse_and_join()
        return out2      
    
    
    def deletePeers(self):
        for key,peer in self.peers.items():
            peer.deletePeer()
        for key in list(peers.keys()):    
            del self.peers[key]

    # create, delete numbered peers
    def createPeers(self,base_name,start=0, end=10):
        for x in range(start,end):
            peername = base_name + '-' + str(x) 
            self.addPeer(peername)

    def deleteNumberedPeers(name):
        for x in range(0,10):
            key = name + str(x)
            self.peers[key].deletePeer()    
            del self.peers[key]
           
    # kubernetes    
    def deleteAllPeers(config):
        for item in config['items']:
            try:
                if 'org_name' in item['data'].keys():
                    self.deletePeer(item['data']['org_name'])
            except Exception as e:
                print(item['data'])    
         
    def installAndAproveChaincode(self):       
        for key, peer in _blockchain.peers.items():
            peer.installAndAproveChaincode(orderer,channel_name,cc_host=cc_host,pkgid=pkgid,cc_hostname=cc_hostname,cc_label=cc_label,shared_fs=shared_fs, sequence=1)        

In [241]:
_blockchain = Blockchain(channel_name='invoicing18')

In [209]:
#_blockchain.peers = {}

## cleanup

In [239]:
_blockchain.deletePeers()
#_blockchain.peers

Error from server (NotFound): deployments.apps "aubonmoulin" not found
Error from server (NotFound): configmaps "hl-fabric-peer-aubonmoulin" not found
Error from server (NotFound): deployments.apps "lyon-impots" not found
Error from server (NotFound): configmaps "hl-fabric-peer-lyon-impots" not found
Error from server (NotFound): deployments.apps "austriafactor" not found
Error from server (NotFound): configmaps "hl-fabric-peer-austriafactor" not found
Error from server (NotFound): deployments.apps "megustaolive" not found
Error from server (NotFound): configmaps "hl-fabric-peer-megustaolive" not found


## init

In [255]:
import json

class Chaincode(object):
    def __init__(self):
        self.shared_fs='kubernetes.research.dev.seeburger.de:30080'
        self.hostname='invoice-net.org1.example.com'
        self.host='10.14.128.38'
        self.service_name='hl-fabric-cc-external-invoice-net'
        self.port = executeKubectlCmd(['get', 'service', cc_service_name, '-o', 'jsonpath="{.spec.ports[?(@.name==\'chaincode\')].nodePort}"'], decode='json')
        self.label='basic_1.0'
        self.pkgid = self.get_pkgid()
        
    def get_pkgid(self):
        strlist = ['curl','-s','--user','scray:scray','http://' + shared_fs + '/cc_descriptions/' + cc_hostname + '_' + cc_label + '/description-hash.json']
        #cmd = toCmd(strlist)
        #output = executePeerCmd(peer.pod_name,  cmd)
        #return json.loads(output[:-1])['description-hash']
        return json.loads(output.decode('ascii'))['description-hash']

In [256]:
Chaincode().__dict__

{'shared_fs': 'kubernetes.research.dev.seeburger.de:30080',
 'hostname': 'invoice-net.org1.example.com',
 'host': '10.14.128.38',
 'service_name': 'hl-fabric-cc-external-invoice-net',
 'port': '32512',
 'label': 'basic_1.0',
 'pkgid': 'basic_1.0:9613e3c481661452e8d5cc9b1d0c0c83316865587edaa7aed3ca0c054ccafae9'}

In [253]:
strlist = ['curl','-s','--user','scray:scray','http://' + shared_fs + '/cc_descriptions/' + cc_hostname + '_' + cc_label + '/description-hash.json']
cmd = toCmd(strlist)
cmd
output = executeLocalCmd(strlist)
output.decode('ascii')['description-hash']

TypeError: string indices must be integers

"[Errno 2] No such file or directory: 'curl -s --user scray:scray http://kubernetes.research.dev.seeburger.de:30080/cc_descriptions/invoice-net.org1.example.com_basic_1.0/description-hash.json'"

In [ ]:
peers = {}
peers

## add channel

In [219]:
_blockchain.add_channel('invoicing18')

('',
 '\x1b2022-01-12 14:36:56.616 UTC [common.tools.configtxgen] main -> INFO 001\x1b Loading configuration\n\x1b2022-01-12 14:36:56.643 UTC [common.tools.configtxgen.localconfig] Load -> INFO 002\x1b Loaded configuration: /mnt/conf/orderer/configtx.yaml\n\x1b2022-01-12 14:36:56.643 UTC [common.tools.configtxgen] doOutputChannelCreateTx -> INFO 003\x1b Generating new channel configtx\n\x1b2022-01-12 14:36:56.652 UTC [common.tools.configtxgen] doOutputChannelCreateTx -> INFO 004\x1b Writing new channel tx\n\x1b2022-01-12 14:36:56.728 UTC [channelCmd] InitCmdFactory -> INFO 001\x1b Endorser and orderer connections initialized\n\x1b2022-01-12 14:36:56.827 UTC [cli.common] readBlock -> INFO 002\x1b Expect block, but got status: &{NOT_FOUND}\n\x1b2022-01-12 14:36:56.831 UTC [channelCmd] InitCmdFactory -> INFO 003\x1b Endorser and orderer connections initialized\n\x1b2022-01-12 14:36:57.033 UTC [cli.common] readBlock -> INFO 004\x1b Expect block, but got status: &{SERVICE_UNAVAILABLE}\n\x1b

In [ ]:
out2.outputs[1]['text'].splitlines()

## create peers

### add peers

In [242]:
_blockchain.addPeer('aubonmoulin', hostname='fr',ca_country='FR',ca_province='Provence-Alpes-Cote d\'Azur',ca_locality='Malaucene')
_blockchain.addPeer('lyon-impots', hostname='gouv.fr',ca_country='FR',ca_province='Auvergne-Rhone-Alpes',ca_locality='Lyon')
_blockchain.addPeer('austriafactor', hostname='at',ca_country='AT',ca_province='Steiermark',ca_locality='Graz')
_blockchain.addPeer('megustaolive', hostname='es',ca_country='ES',ca_province='Andalusia',ca_locality='Dos Hermanas')

In [235]:
_blockchain.peers

{'aubonmoulin': <__main__.KubernetesPeer at 0x7f1ce3a2f070>,
 'lyon-impots': <__main__.KubernetesPeer at 0x7f1ce0ba1280>,
 'austriafactor': <__main__.KubernetesPeer at 0x7f1d201aebe0>,
 'megustaolive': <__main__.KubernetesPeer at 0x7f1ce285f3d0>}

In [ ]:

peers={}    
addPeerFromKubernetes('aubonmoulin')
addPeerFromKubernetes('lyon-impots')
addPeerFromKubernetes('austriafactor')
addPeerFromKubernetes('megustaolive') 

peernames = list(peers.keys())

### endorse and join

In [100]:
## endorse and join
import ipywidgets as widgets

#orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host

In [ ]:
_blockchain.endorse_and_join_all_peers()

## install chaincode

In [ ]:
for key, peer in _blockchain.peers.items():
    executePeerCmd(peer.pod_name,'apk add curl')
    strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh', '-o', '/mnt/conf/install_and_approve_cc.sh']
    cmd = toCmd(strlist)
    print(cmd)
    output = executePeerCmd(peer.pod_name,  cmd)
    executePeerCmd(peer.pod_name,'chmod 755 /mnt/conf/install_and_approve_cc.sh')
    strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/peer/cc_commit.sh', '-o', '/mnt/conf/peer/cc_commit.sh']
    cmd = toCmd(strlist)
    print(cmd)
    output = executePeerCmd(peer.pod_name,  cmd)
    executePeerCmd(peer.pod_name,'chmod 755 /mnt/conf/peer/cc_commit.sh')

In [ ]:
for key, peer in _blockchain.peers.items():
    peer.installAndAproveChaincode(orderer,channel_name,cc_host=cc_host,pkgid=pkgid,cc_hostname=cc_hostname,cc_label=cc_label,shared_fs=shared_fs, sequence=1)

In [146]:
#peernames = list(peers.keys())
#peernames
peer = peers[peernames[0]]
commitChaincode(peer,channel_name,sequence=1,pkgid=pkgid)

/mnt/conf/peer/cc_commit.sh invoicing17 1 basic_1.0:9613e3c481661452e8d5cc9b1d0c0c83316865587edaa7aed3ca0c054ccafae9
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-79666dbdd9-wpjb5 -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/cc_commit.sh invoicing17 1 basic_1.0:9613e3c481661452e8d5cc9b1d0c0c83316865587edaa7aed3ca0c054ccafae9


2022-01-10 14:30:16.140 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [7551f8d142d95a3bce8e3c81eded075f7f35cf7783c03583af2b9f5592d271c7] committed with status (VALID) at peer0.aubonmoulin.fr:32432


'Installed chaincode\nInstalled chaincodes on peer:\nPackage ID: basic_1.0:9613e3c481661452e8d5cc9b1d0c0c83316865587edaa7aed3ca0c054ccafae9, Label: basic_1.0\n'